In [4]:
!pip install -q openai langchain pyvis gradio transformers newsapi-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 6

In [7]:
import pandas as pd
import numpy as np
from newsapi.newsapi_client import NewsApiClient
import gradio as gr

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

## **Financial Sentiment Analysis**
We download news articles using newsapi Python client related to the Eurozone or EUR currency from the NewsAPI portal. For this, I wrote a method that fetches news articles for the specified date. Following search terms are used in the query:

  * "European Commission"
  * "EU"
  * "European Central Bank"
  * "ECB"
  * "Eurozone"
  * "EUR currency".

Then, I used a transformer language model (LLM) from HuggingFace's transformers library to generate sentiment scores for the downloaded articles. The sentiments for each article's title and content are then calculated and added to the DataFrame.

Finally, I'll extend it using the gradio library such that I can take input a headline of article via interface and predict the sentiment using the LLM.

In [73]:
# Initialize the NewsAPI client with your API key
newsapi = NewsApiClient(api_key=<YOUR_API_KEY>)

In [74]:
# Define a function to fetch news and generate sentiments
def fetch_news_and_sentiment(query, from_date, to_date, pipeline):
  # Fetch news articles
  all_articles = newsapi.get_everything(q=query,
  from_param=from_date,
  to=to_date,
  language='en',
  sort_by='publishedAt')

  # Generate sentiments for the articles
  sentiments = []
  for article in all_articles['articles']:
    sentiment = pipeline(article['title'])
    sentiments.append({
    'date': article['publishedAt'],
    'title': article['title'],
    'sentiment': sentiment[0]['label']
    })

  return sentiments

In [75]:
# Load the tokenizer and model from Hugging Face's model repository
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Create a sentiment analysis pipeline using the specific model
sentiment_analysis = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [76]:
sentiments = fetch_news_and_sentiment('"European Commission" OR "EU" OR "European Central Bank" OR "ECB" OR "Eurozone" OR "EUR currency"','2024-04-01','2024-04-19', sentiment_analysis)
for sentiment in sentiments:
  print(sentiment)

{'date': '2024-04-19T23:33:23Z', 'title': "Blinken: Military operation in Rafah would have 'terrible consequences' for civilians", 'sentiment': 'NEGATIVE'}
{'date': '2024-04-19T23:28:00Z', 'title': 'Football: Vitesse docked 18 points amid Russia finance probe', 'sentiment': 'NEGATIVE'}
{'date': '2024-04-19T23:14:48Z', 'title': 'Risks Digest 34.18', 'sentiment': 'NEGATIVE'}
{'date': '2024-04-19T23:05:24Z', 'title': 'Irish GPs have come under pressure to prescribe hormones, gender service consultant claims', 'sentiment': 'NEGATIVE'}
{'date': '2024-04-19T23:00:36Z', 'title': 'World Leaders Urge Restraint After Israel Attacks Iran', 'sentiment': 'NEGATIVE'}
{'date': '2024-04-19T22:57:00Z', 'title': 'Apple’s offer to open up NFC reportedly to be approved by EU next month', 'sentiment': 'POSITIVE'}
{'date': '2024-04-19T22:55:00Z', 'title': 'Apple Users in Europe Get a New Third-Party App Store - CNET', 'sentiment': 'NEGATIVE'}
{'date': '2024-04-19T22:41:38Z', 'title': 'Scholz Visit to China 

### **Quick LLM-based Web App**
I extend this using the gradio library such that I can take input a headline of article via interface and predict the sentiment using the LLM.

In [77]:
# Define a function for the Gradio interface to predict sentiment
def predict_sentiment(headline):
  sentiment = sentiment_analysis(headline)
  return sentiment[0]['label']

# Create the Gradio interface
iface = gr.Interface(
        fn=predict_sentiment,
        inputs=gr.Textbox(lines=2, placeholder="Enter a news headline here..."),
        outputs=gr.Label(),
        title="Financial Sentiment Analysis!",
        description="This interface predicts the financial sentiment of a news headline using a transformer model."
)
# Run the Gradio interface
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://706ed2a29a1980b0a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
